# Prepare English Data splits

- from fineweb and fineweb-edu
- use subset of fineweb for tests (random)
- use fineweb-edu for training (ensure there is no overlap with the fineweb test set)

In [2]:
from datasets import load_dataset


fineweb = load_dataset("HuggingFaceFW/fineweb", "sample-10BT", split="train")
fineweb_edu = load_dataset("HuggingFaceFW/fineweb-edu", "sample-10BT", split="train")

Resolving data files:   0%|          | 0/25868 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/102 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2110 [00:00<?, ?it/s]

000_00000.parquet:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

001_00000.parquet:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

...

011_00000.parquet:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

012_00000.parquet:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

013_00000.parquet:   0%|          | 0.00/541M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9672101 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/98 [00:00<?, ?it/s]

In [5]:
test_size = 26631 #test size to match Czech dataset size for perplexity evaluation

In [ ]:
fineweb_split = fineweb.train_test_split(test_size=test_size, shuffle=True, seed=42)



In [9]:
fineweb_split.save_to_disk("/mnt/personal/mlynatom/data/pretraining/fineweb_train_test_split")

Saving the dataset (0/102 shards):   0%|          | 0/14842231 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/26631 [00:00<?, ? examples/s]

In [12]:
#select fineweb_edu subset of 500k

fineweb_edu.shuffle(seed=42)
fineweb_edu_subset = fineweb_edu.take(500000)

In [13]:
fineweb_edu_subset

Dataset({
    features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
    num_rows: 500000
})

In [17]:
ids = set(fineweb_split['test']['id'])

In [20]:
#filter the fields with the same ids from fineweb_edu
def filter_fn(example):
    return example['id'] not in ids

fineweb_edu_filtered_subset = fineweb_edu.filter(filter_fn)

Filter:   0%|          | 0/9672101 [00:00<?, ? examples/s]

In [21]:
fineweb_edu_filtered_subset

Dataset({
    features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
    num_rows: 9672093
})

In [23]:
fineweb_edu_filtered_subset.save_to_disk("/mnt/personal/mlynatom/data/pretraining/fineweb_edu_filtered")

Saving the dataset (0/99 shards):   0%|          | 0/9672093 [00:00<?, ? examples/s]

## Prepare mixture for training

In [18]:
from datasets import load_from_disk, load_dataset, concatenate_datasets

SEED = 42

In [2]:
one_split = 500000 / 4
cs_size = 3*one_split
en_size = one_split

In [ ]:
dataset_cs = load_dataset("HuggingFaceFW/fineweb-2", "ces_Latn", split="train")
dataset_cs = dataset_cs.remove_columns(["id", "dump", "url", "date", "file_path", "language", "language_score", "language_script", "minhash_cluster_size", "top_langs"])
dataset_cs = dataset_cs.shuffle(seed=SEED)


In [7]:
dataset_cs = dataset_cs.take(int(cs_size))

In [8]:
dataset_cs

Dataset({
    features: ['text'],
    num_rows: 375000
})

In [10]:
dataset_en = load_from_disk("/mnt/personal/mlynatom/data/pretraining/fineweb_edu_filtered")

Loading dataset from disk:   0%|          | 0/99 [00:00<?, ?it/s]

In [14]:
dataset_en = dataset_en.remove_columns(["id", "dump", "url", "file_path", "language", "language_score", "token_count", "score", "int_score"])
dataset_en = dataset_en.shuffle(seed=SEED)
dataset_en = dataset_en.take(int(en_size))

In [19]:
#join the datasets
dataset = concatenate_datasets([dataset_cs, dataset_en])
dataset = dataset.shuffle(seed=SEED)

In [20]:
dataset

Dataset({
    features: ['text'],
    num_rows: 500000
})

In [21]:
dataset.save_to_disk("/mnt/personal/mlynatom/data/pretraining/fineweb2-cs_finewebedu-en_31_500k")

Saving the dataset (0/4 shards):   0%|          | 0/500000 [00:00<?, ? examples/s]